In [1]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, SimpleRNN, LSTM
from keras.utils import np_utils
import numpy as np
import glob

Saving 3D Arrays:
https://www.geeksforgeeks.org/how-to-load-and-save-3d-numpy-array-to-file-using-savetxt-and-loadtxt-functions/

In [2]:
#Loads train/test data from .txt files to np arrays 
train_reload = np.loadtxt("jul19data/train_data.txt")
test_reload = np.loadtxt("jul19data/test_data.txt")

train_label = np.loadtxt('jul19data/train_label.txt').astype(int)
test_label = np.loadtxt('jul19data/test_label.txt').astype(int)

print("Loaded data files...")

(2830,)
Loaded data files...


In [13]:
train_data = np.reshape(train_reload,(-1,50, 8))
test_data = np.reshape(test_reload,(-1,50, 8))

In [14]:
train_label = np.loadtxt('jul19data/train_label.txt').astype(int)
test_label = np.loadtxt('jul19data/test_label.txt').astype(int)

A = sum(train_label == 1)
B = sum(test_label == 1)
train_label = np.zeros((train_label.shape-A)*10)
train_label = np.append(train_label,np.ones(A*10))
test_label = np.zeros((test_label.shape-B)*10)
test_label = np.append(test_label,np.ones(B*10))
print(train_label.shape)
print(test_label.shape)

(28300,)
(9900,)


In [15]:
#Build the model
model = Sequential()
model.add(LSTM(24, input_shape = (50,8), activation = 'tanh', recurrent_activation='sigmoid', return_sequences=True))
model.add(LSTM(12, activation = 'tanh', recurrent_activation='sigmoid', return_sequences=True))
model.add(LSTM(4, activation = 'tanh', recurrent_activation='sigmoid'))
model.add(Dense(1, activation='sigmoid'))
# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_6 (LSTM)                (None, 50, 24)            3168      
_________________________________________________________________
lstm_7 (LSTM)                (None, 50, 12)            1776      
_________________________________________________________________
lstm_8 (LSTM)                (None, 4)                 272       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 5         
Total params: 5,221
Trainable params: 5,221
Non-trainable params: 0
_________________________________________________________________


In [16]:
model.fit(train_data, train_label, validation_data=(test_data, test_label), epochs=20, batch_size=500)

Epoch 1/20
57/57 [==============================] - 8s 41ms/step - loss: 0.6015 - accuracy: 0.8134 - val_loss: 0.4629 - val_accuracy: 0.8668
Epoch 2/20
57/57 [==============================] - 1s 18ms/step - loss: 0.4338 - accuracy: 0.8798 - val_loss: 0.3789 - val_accuracy: 0.8887
Epoch 3/20
57/57 [==============================] - 1s 18ms/step - loss: 0.3643 - accuracy: 0.8913 - val_loss: 0.3362 - val_accuracy: 0.8951
Epoch 4/20
57/57 [==============================] - 1s 18ms/step - loss: 0.3275 - accuracy: 0.8984 - val_loss: 0.3128 - val_accuracy: 0.9051
Epoch 5/20
57/57 [==============================] - 1s 17ms/step - loss: 0.3021 - accuracy: 0.9082 - val_loss: 0.2972 - val_accuracy: 0.9084
Epoch 6/20
57/57 [==============================] - 1s 17ms/step - loss: 0.2913 - accuracy: 0.9098 - val_loss: 0.2904 - val_accuracy: 0.9081
Epoch 7/20
57/57 [==============================] - 1s 17ms/step - loss: 0.2791 - accuracy: 0.9141 - val_loss: 0.2823 - val_accuracy: 0.9061
Epoch 8/20
57

In [9]:
test_accuracy = model.evaluate(test_data, test_label)

120/120 [==============================] - 2s 17ms/step - loss: 0.3181 - accuracy: 0.8819


In [22]:
#Load prediction data
pred_load = np.loadtxt("jul19data/pred_data.txt")
pred_data = np.reshape(pred_load,(-1,50, 8))
pred_label = np.loadtxt("jul19data/pred_label.txt")
A = sum(pred_label == 1)
pred_label = np.zeros((pred_label.shape-A)*10)
pred_label = np.append(pred_label,np.ones(A*10))

In [23]:
model.evaluate(pred_data, pred_label)

310/310 [==============================] - 2s 6ms/step - loss: 0.1604 - accuracy: 0.9365


[0.16043104231357574, 0.9364646673202515]

In [9]:
normal_load = np.loadtxt("normal/61.44.csv", delimiter = ',')
# normal_load = np.loadtxt("imbalance/35g/56.7296.csv", delimiter = ',')
normal_data = normal_load[200000:250000,:]
print(normal_data.shape)

(50000, 8)


In [10]:
files_no = glob.glob('normal/*.csv')
# errors = numpy.empty(1)
for f_on in files_no:
    normal_data = np.loadtxt(f_on, delimiter=",")
    normal_data = normal_data.reshape(-1,50,8)
    y = model.predict_classes(normal_data)
    print(f_on, sum(y)/y.shape[0])



C:\Users\Amir\anaconda3\envs\tf-gpu-new\lib\site-packages\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


normal\12.288.csv [0.532]
normal\13.1072.csv [0.1448]
normal\14.336.csv [0.0534]
normal\15.1552.csv [0.0546]
normal\16.1792.csv [0.269]
normal\17.2032.csv [0.3896]
normal\18.432.csv [0.0736]
normal\19.6608.csv [0.1546]
normal\20.2752.csv [0.2498]
normal\21.7088.csv [0.1638]
normal\22.3232.csv [0.0622]
normal\23.552.csv [0.0764]
normal\24.576.csv [0.1246]
normal\25.6.csv [0.1634]
normal\26.624.csv [0.104]
normal\27.4432.csv [0.1232]
normal\28.8768.csv [0.0706]
normal\29.4912.csv [0.186]
normal\30.72.csv [0.1014]
normal\31.744.csv [0.0428]
normal\32.9728.csv [0.0478]
normal\33.5872.csv [0.0586]
normal\34.2016.csv [0.0804]
normal\35.4304.csv [0.0368]
normal\36.4544.csv [0.035]
normal\37.6832.csv [0.0402]
normal\38.2976.csv [0.0378]
normal\39.3216.csv [0.028]
normal\40.3456.csv [0.045]
normal\41.7792.csv [0.0428]
normal\42.3936.csv [0.0502]
normal\43.6224.csv [0.0474]
normal\44.6464.csv [0.0422]
normal\45.4656.csv [0.0502]
normal\46.2848.csv [0.0518]
normal\47.7184.csv [0.0716]
normal\48.9

In [10]:
#Load prediction data
pred_load = np.loadtxt("pred_data.txt")
pred_label = np.zeros(490*500)
pred_label = np.append(pred_label, np.ones(3330*500), axis=0)

In [11]:
pred_data = np.reshape(pred_load,(-1, 1, 8))
model.evaluate(pred_data, pred_label)

59688/59688 [==============================] - 197s 3ms/step - loss: 0.6386 - accuracy: 0.8145


[0.6385664343833923, 0.8144764304161072]

In [12]:
print(pred_data[0,0])

[-0.56815   0.60986   0.034358  0.033138 -0.63896   0.016935 -0.17671
  0.022027]
